### Importing all the relevant libraries and check the installed package versions

In [1]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections as mc
import pkg_resources
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import sobol_seq
import time
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for rq in requirements:
    print("{}=={}".format(*rq))

seaborn==0.10.1
scipy==1.5.0
pandas==1.0.5
numpy==1.18.5
matplotlib==3.2.2


## Import the EC payments database

In [ ]:
df = pd.read_excel('Database_Final_UPD_2020_corr.xlsx')

### Generate the series of quasi-random numbers for the analysis

In [ ]:
quasiRandom_df = pd.DataFrame(sobol_seq.i4_sobol_generate(4,2**17))

DistributionFiMax = 0.8+quasiRandom_df[0]*0.2
DistributionFiMin = 0.2+quasiRandom_df[1]*0.2

Residue_selector = quasiRandom_df[2].round(0).astype(int)

inputs = pd.concat([DistributionFiMax, DistributionFiMin, Residue_selector],axis=1)

### Rearrange the dataframe for our convenience

In [ ]:
dfb = df[['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year','CF_TOTAL','EAGGF','ERDF_TOTAL','ESF']].copy()
df2 = dfb.melt(id_vars=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year'],var_name='FundingScheme')
df2b = df2[df2.Year<2018]
df3 = df2.pivot_table(index=['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code'],columns='Year', values='value')
df4 = df3.dropna(how='all').fillna(0)

### Get the slice corresponding to the set we'll be working on

In [ ]:
df4b = df4.loc[('2007-2013','ERDF_TOTAL')]

df4b.loc[pd.IndexSlice[:,:,'ITH1'],:]=df4b.loc[pd.IndexSlice[:,:,'ITH1'],:].values+\
df4b.loc[pd.IndexSlice[:,:,'ITH2'],:].values
df4b.loc[pd.IndexSlice[:,:,'ITH1'],:]
df4b=df4b.reset_index()
df4b.NUTS2Code[df4b.NUTS2Code=='ITH1']='ITH0'
df4b = df4b[df4b.NUTS2Code != 'ITH2']
df4b=df4b.set_index(['Country','NUTS1Code','NUTS2Code']).sort_index()

### Excluding years before the commencing of the programming periods

In [ ]:
dummy = []

diff = int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6-\
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4])

d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in \
range(int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4]),
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6)])

dummy.append(d_Var)

for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[1:-1]:
    diff = int(pp[5:])+4-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+4)])
    
    dummy.append(d_Var)
    
for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[-1:]:
    diff = int(pp[5:])+5-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+5)])
    
    dummy.append(d_Var)

### Normalise the cumulative figures

In [ ]:
df5 = df4b.copy()

Norm_df6 = ((df5.cumsum(axis=1).T/df5.cumsum(axis=1).max(axis=1).values).T).dropna(how='all')

### Define the outcome variables

In [ ]:
Norm_df6['$\phi$']=(Norm_df6.loc[:,[y for y in range(2007,2018)]]-dummy[-1]).cumsum(axis=1).iloc[:,-1]

Norm_df6['$\mu$']=(Norm_df6['$\phi$'].max()-Norm_df6['$\phi$'])/(Norm_df6['$\phi$'].max()-Norm_df6['$\phi$'].min())

### Set the trigger for the number of years the residual expenditure gets spread onto on the last eligible year of the programming period

In [ ]:
ld1 = []
for iq,qr in enumerate(quasiRandom_df[3]):
    df8b = Norm_df6.copy()
    df8b[0]=(qr*(Norm_df6['$\mu$']*(len(dummy[-1])-1)).astype(int)).astype(int)+1
    for il in range(1,len(dummy[-1])):
        df8b[il]=df8b[0]-il
    df8b[df8b<1]=1
    cd = [il0 for il0 in range(len(dummy[-1]))]
    df8b['value']=iq
    cd.append('value')
    df8b = df8b[cd]
    ld1.append(df8b)
years = pd.concat(ld1)
years.set_index('value', append=True, inplace=True)

### Define the yearly residues

In [ ]:
def A(n):
    return [2**j/(2**n-1) for j in reversed(range(n))]

B9 = []
for k in reversed(range(2,12)):
    B9.append(pd.DataFrame([A(y) for y in range(1,k)],index=[y for y in range(1,k)],
                           columns=[y for y in range(1,k)]).fillna(0).sort_values(by=1,ascending=False))

### And the number of years from which the Excess get taken out

In [ ]:
def C(n):
    return [1/n for j in reversed(range(n))]

C9 = pd.DataFrame([C(y) for y in range(1,11)],index=[y for y in range(1,11)],
                           columns=[y for y in range(1,11)]).fillna(0).sort_values(by=1,ascending=False)

D = [C9,pd.DataFrame([A(y) for y in range(1,11)],index=[y for y in range(1,11)],
                           columns=[y for y in range(1,11)]).fillna(0).sort_values(by=1,ascending=False)]

## Get the EC payment slice for Italy

In [ ]:
Payments = df4b.loc['IT'].iloc[:,-12:].copy()
Payments.index=Payments.index.droplevel(0)

### Get the member-state figure our data will be compared against

In [ ]:
df1 = pd.read_excel('20181231 Pagamenti ammessi PO 2007-2013.xls',usecols=[0,1,2,3,4,5,7])
df_REGIO_capped = df1[(df1['CCI'].str.contains("161"))|df1['CCI'].str.contains("162")]
NUTS2_dic = {'ABRUZZO':'ITF1','BASILICATA':'ITF5','CALABRIA':'ITF6','CAMPANIA':'ITF3','EMILIA':'ITH5','FRIULI':'ITH4','LAZIO':'ITI4',
'LIGURIA':'ITC3','LOMBARDIA':'ITC4','MARCHE':'ITI3','MOLISE':'ITF2','PIEMONTE':'ITC1','PUGLIA':'ITF4','SARDEGNA':'ITG2','SICILIA':'ITG1',
'TOSCANA':'ITI1','TRENTINO':'ITH0','UMBRIA':'ITI2',"VALLE D'AOSTA":'ITC2','VENETO':'ITH3'}

df_REGIO_capped['NUTS2'] = df_REGIO_capped.REGIONE.map(NUTS2_dic)

df_REGIO_capped.DESCRIZIONE_PROGRAMMA = df_REGIO_capped.DESCRIZIONE_PROGRAMMA.str.replace(' Romagna', '')

### Attribute the orphan payments

In [ ]:
df_REGIO_capped.loc[df_REGIO_capped['NUTS2'].isnull(),'NUTS2'] = \
df_REGIO_capped['DESCRIZIONE_PROGRAMMA'].str.split(expand=True)[3].map(NUTS2_dic)

df_REGIO_nonAttributed = df_REGIO_capped[df_REGIO_capped.NUTS2.isna()]
df_REGIO_REGIO = df_REGIO_capped.dropna()

df_REGIO_REGIO=df_REGIO_REGIO.rename(columns={'ANNO':'Year'})

df_REGIO_REGIO_yearly=df_REGIO_REGIO.groupby(['Year','NUTS2']).sum()
df_REGIO_REGIO_yearly['Year']=df_REGIO_REGIO_yearly.index.get_level_values(0)
df_REGIO_REGIO_yearly=df_REGIO_REGIO_yearly
df_REGIO_REGIO_yearly.index= df_REGIO_REGIO_yearly.index.droplevel(0)

df_REGIO_pv = df_REGIO_REGIO_yearly.pivot_table(index='Year', columns='NUTS2', values='PAGAMENTO_AMMESSO_UE').fillna(0).T

### Re-distribute the residues as per the NUTS2 expenditures

In [ ]:
df_REGIO_multiplier_year=df_REGIO_nonAttributed.groupby('ANNO').sum().T.values*df_REGIO_pv/df_REGIO_pv.sum()

df_REGIO_multiplier_year['r']=1

df_REGIO_multiplier = df_REGIO_nonAttributed.groupby('ANNO').sum().T.values*\
pd.concat([df_REGIO_pv.sum(axis=1)/df_REGIO_pv.sum(axis=1).sum() for re in range(12)],axis=1)
df_REGIO_multiplier['r']=0
df_REGIO_multiplier.columns = df_REGIO_multiplier_year.columns

df_REGIO_residual = pd.concat([df_REGIO_multiplier,df_REGIO_multiplier_year])

### Evaluate the distance between the modelled expenditure and the MS incurred

In [38]:
Expe = pd.concat([Payments.copy() for r in range(len(quasiRandom_df))])

N_df = pd.concat([Norm_df6.loc['IT','$\mu$'] for r in range(len(quasiRandom_df))])
N_df.index = N_df.index.droplevel(0) 

Est_expenditure = pd.concat([df_REGIO_pv.copy() for r in range(len(quasiRandom_df))])

Exp = Expe.copy()

Exp['r']=np.array([r for r in range(len(quasiRandom_df)) for er in range(len(Payments))])

Exp.loc[:,2017]=Expe.loc[:,[y for y in range(2017,2019)]].sum(axis=1)
Exp.loc[:,[y for y in range(2018,2019)]]=0

Aggregate = Exp.loc[:,2017]*(1-(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-\
                                                                      DistributionFiMin[Exp.r]).values))

Exp.loc[:,2017] = Exp.loc[:,2017]*(DistributionFiMax[Exp.r].values-N_df*
                            (DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)

y_slice = years.loc[pd.IndexSlice['IT',:,:]]
y_slice.index = y_slice.index.droplevel(0)


for iy2,y2 in enumerate(reversed(range(2007,2017))):
    Exp[y2] = Expe[y2]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)

    for iy3,y3 in enumerate(reversed(range(y2,2016))):
        Exp[y2]+=Expe[y3+1]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-\
                DistributionFiMin[Exp.r]).values)*B9[iy3].loc[y_slice.iloc[:,iy3].values,(y3+1-y2)].values

    Exp[y2] += Aggregate*B9[0].loc[y_slice.iloc[:,0],iy2+1].values

Exp[2007] += Expe[2007]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)
Exp = Exp.drop('r',axis=1)

Distance_fixed_residual = []

for ie in range(2):
    residual = pd.concat(df_REGIO_residual.loc[df_REGIO_residual.r==ie] for rs in Residue_selector)

    MS_Expenditure = (Est_expenditure+residual).drop('r',axis=1)

    Distance = pd.DataFrame((np.abs(MS_Expenditure.cumsum(axis=1)-Exp.cumsum(axis=1)).sum(axis=1)/\
                             Expe.cumsum(axis=1).iloc[:,-1])).rename(columns={2016:'Distance'})

    Distance['r']=np.array([r for r in range(len(quasiRandom_df)) for er in range(len(Payments))])

    Distance_df = Distance.pivot_table(values=0,index='r',columns='NUTS2')
    
    Distance_fixed_residual.append(Distance_df)

In [39]:
dict_code = dict(zip(Payments.index, ['Pdm','AsV','Lgr','Lmb','Abr','Mls','Cmp','Apl','Bsl','Clb','Scl','Srd','Vnt','FVG','EmR','TST','Tsc',
                 'Umb','Mrc','Laz']))

Dfs = Distance_fixed_residual.copy()
for d in Dfs:
    d.loc['code'] = d.columns.map(dict_code.get)
    d.columns = d.loc['code']
    
final_comparison = pd.concat([pd.concat(Dfs,axis=1).iloc[:-1], 
                              pd.read_excel('2020-06-15_distance_modelled_observed_expenditures.xlsx', index_col=0)],
                             axis=1)

### Uncertainty analysis - distribution plots

In [ ]:
for c in final_comparison.columns.get_level_values(0).unique():
    plt.figure(figsize=(16, 10), dpi= 300, facecolor='w', edgecolor='k')
    ls = [y for y in range(1,len(final_comparison[c].columns))]
    final_comparison[c].plot(kind='box')
    ls.extend(['Var'])
    plt.xticks(np.arange(1,len(final_comparison[c].columns)+1),
              tuple(ls),fontsize = 18)
    plt.ylabel('Distance',fontsize = 24)
    plt.xlabel('Residual',fontsize = 24)
    plt.tight_layout()
    plt.show()
#     plt.savefig(str(c)+'_residual_fixing.png', dpi= 300)
#     plt.close()